In [4]:
import pickle
import numpy as np
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from sklearn.preprocessing import LabelEncoder
from proglearn.forest import UncertaintyForest
import time
from collections import Counter
import tensorflow as tf

In [5]:
(X, Y), (X_test_org, Y_test_org) = pickle.load(open('/home/weiwya/cifar_100_Bit_m-r101x1_embd.p', 'rb'))
print(X.shape, X_test_org.shape)



(50000, 2048) (10000, 2048)


In [6]:
def gen_fine_level_data(X, fine_label, coarse_label,  X_test,  fine_label_test, coarse_label_test, wanted_idx):
    selected = np.where(coarse_label==wanted_idx)[0]
    X_selected = X[selected]
    Y_fine_selected = fine_label[selected]
    le = LabelEncoder().fit(Y_fine_selected)
    Y_fine_selected = le.transform(Y_fine_selected)
    
    selected_test = np.where(coarse_label_test == wanted_idx)[0]
    X_test_selected = X_test[selected_test]
    Y_fine_test_selected = fine_label_test[selected_test]
    Y_fine_test_selected = le.transform(Y_fine_test_selected)
    return X_selected, Y_fine_selected, X_test_selected, Y_fine_test_selected, le


def gen_fine_clf (X, Y_coarse, Y_fine, X_test, Y_coarse_test, Y_fine_test, wanted_idx, n_estimators=100, max_depth=10):
    X_fine_train, Y_fine_train, X_fine_test, Y_fine_test, le = gen_fine_level_data(X, Y_fine, Y_coarse, X_test, Y_fine_test, Y_coarse_test, wanted_idx )
    
    print(X_fine_train.shape, X_fine_test.shape)
    clf_fine = UncertaintyForest(n_estimators=n_estimators,
                        max_depth=max_depth)
    clf_fine.fit(X_fine_train, Y_fine_train)
    
#     pp = clf_fine.predict(X_fine_test, task_id=0)    
#     print(classification_report( Y_fine_test, pp))
    
    return clf_fine, le

def eval_hieracy_cluster_one_step(data, coarse_clf, fine_clfs):
    corse_level = coarse_clf.predict(data)
    res = []
    for i, c in enumerate(corse_level):
        fine_clf, le = fine_clfs[c]
        p = fine_clf.predict(data[i].reshape(1,-1))
        res.append(le.inverse_transform(p)[0])

    return res

def eval_hieracy_cluster_two_step(data, coarse_clf, fine_clfs):
    #get Corase level probality
    print('done coarse %s ' %(data.shape[0]))
    coarse_prob = coarse_clf.predict_proba(data)
    fine_probs, fine_res = [],[]
    
    for idx, (fine_clf, le) in fine_clfs.items():
        fine_prob = fine_clf.predict_proba(data)
        classes = fine_prob.argmax(axis=1)
        print(classes.shape)
        classes = le.inverse_transform(classes)
        #baysian
        fine_prob = coarse_prob[:, idx].reshape(-1,1) * fine_prob
        
        fine_prob = np.max(fine_prob, axis=1)
        fine_probs.append(fine_prob)
        fine_res.append(classes)
        print('done fine %s' %idx)

        
    fine_probs  = np.array(fine_probs)
    wanted  = fine_probs.argmax(axis=0)
    res = []
    for idx, w in enumerate(wanted):
        res.append(fine_res[w][idx])
    return res

def hieracy_cluster(X_train, Y_train, X_test, Y_test, used_hierarchy, n_estimators=100, max_depth=10 ):
    #relabel classes to hierarchy labels
    hierarchy_relabel = {i:used_hierarchy[i] for i in range(used_hierarchy.shape[0])    }
    
    #change fine label to corse label
    Y_train_hierarchy = np.array([hierarchy_relabel[x[0]] for x in Y_train])
    Y_test_hierarchy = np.array([hierarchy_relabel[x[0]] for x in Y_test])
    
    clf_coarse = UncertaintyForest(n_estimators=n_estimators,
                        max_depth=max_depth)
    clf_coarse.fit(X_train, Y_train_hierarchy)
    
    print('done train corase labels')
    
    fine_clfs= {}
    for wanted_idx in np.unique(used_hierarchy):
        print('processing %s' %wanted_idx)
        fine_clf, le = gen_fine_clf(X_train, Y_train_hierarchy, Y_train, X_test, Y_test_hierarchy, 
                                    Y_test, wanted_idx, n_estimators=n_estimators, max_depth=max_depth )
        fine_clfs[wanted_idx] = (fine_clf, le)
    
    return clf_coarse, fine_clfs

In [15]:
#match up fine/coarse label and geneate look-ups
(_, Y_train_fine), (_, Y_test_fine) = tf.keras.datasets.cifar100.load_data(label_mode='fine')
(_, Y_train_coarse), (_, Y_test_coarse) = tf.keras.datasets.cifar100.load_data(label_mode='coarse')
used_clusters = {}
for fine, coarse in zip(Y_train_fine, Y_train_coarse):
    used_clusters[fine[0]] = coarse[0]

used_clusters=sorted([(k,v) for k, v in used_clusters.items()], key=lambda x : x[0])
used_clusters = [x[1] for x in used_clusters]
print(used_clusters)
used_clusters = np.array(used_clusters)

[4, 1, 14, 8, 0, 6, 7, 7, 18, 3, 3, 14, 9, 18, 7, 11, 3, 9, 7, 11, 6, 11, 5, 10, 7, 6, 13, 15, 3, 15, 0, 11, 1, 10, 12, 14, 16, 9, 11, 5, 5, 19, 8, 8, 15, 13, 14, 17, 18, 10, 16, 4, 17, 4, 2, 0, 17, 4, 18, 17, 10, 3, 2, 12, 12, 16, 12, 1, 9, 19, 2, 10, 0, 1, 16, 12, 9, 13, 15, 13, 16, 19, 2, 4, 6, 19, 5, 5, 8, 19, 18, 1, 2, 15, 6, 0, 17, 8, 14, 13]


In [16]:
n_iter = 10
n_trees = 100
max_depth =10
clfs = []
accuracies_1step = []
accuracies_2step = []

master_seed = 42
np.random.seed(master_seed)
seeds = np.random.randint(10000, size=n_iter)




for iteration in range(n_iter):
    start = time.time()
    seed =  seeds[iteration]
    print(iteration)
    X_train, _, Y_train, _ = train_test_split(X, Y, test_size=0.9, random_state=seed)
    print(X_train.shape, X_test_org.shape)
        
    clf_coarse, clf_fines = hieracy_cluster(X_train, Y_train, X_test_org, Y_test_org, used_clusters, n_estimators=n_trees, 
                                            max_depth=max_depth)
    
    
    
    clfs.append((clf_coarse, clf_fines))
    pp = eval_hieracy_cluster_one_step(X_test_org, clf_coarse, clf_fines)
    
    accuracies_1step.append(accuracy_score(Y_test_org, pp))
    
    pp = eval_hieracy_cluster_two_step(X_test_org, clf_coarse, clf_fines)
    accuracies_2step.append(accuracy_score(Y_test_org, pp))

    
    print(accuracies_1step[-1], accuracies_2step[-1])
    print('took %s' %(time.time() - start))
    print()

with open('/home/weiwya/cirar_truth_10_10_run.p', 'wb') as handle:
    pickle.dump((clfs, accuracies_1step, accuracies_2step), handle)

print(np.average(accuracies_1step), np.average(accuracies_2step))
 
    
    
        

0
(5000, 2048) (10000, 2048)
done train corase labels
processing 0
(254, 2048) (500, 2048)


/home/weiwya/screen_understanding/screen_env/lib/python3.7/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


processing 1
(229, 2048) (500, 2048)


/home/weiwya/screen_understanding/screen_env/lib/python3.7/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


processing 2
(248, 2048) (500, 2048)


...

processing 18
(237, 2048) (500, 2048)


/home/weiwya/screen_understanding/screen_env/lib/python3.7/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


processing 19
(269, 2048) (500, 2048)


/home/weiwya/screen_understanding/screen_env/lib/python3.7/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


done coarse 10000 
(10000,)
done fine 0
(10000,)
done fine 1
(10000,)
done fine 2
(10000,)
done fine 3
(10000,)
done fine 4
(10000,)
done fine 5
(10000,)
done fine 6
(10000,)
done fine 7
(10000,)
done fine 8
(10000,)
done fine 9
(10000,)
done fine 10
(10000,)
done fine 11
(10000,)
done fine 12
(10000,)
done fine 13
(10000,)
done fine 14
(10000,)
done fine 15
(10000,)
done fine 16
(10000,)
done fine 17
(10000,)
done fine 18
(10000,)
done fine 19
0.4743 0.5311
took 1206.431851863861

0.4675000000000001 0.52854


In [17]:
np.average(accuracies_2step)

0.52854

In [18]:
np.std(accuracies_2step)/np.sqrt(10)

0.0016865467678069315

In [19]:
print("Truth average:%s std_err: %s " %(np.average(accuracies_2step), np.std(accuracies_2step)/np.sqrt(n_iter)   ))


Truth average:0.52854 std_err: 0.0016865467678069315 
